# Udacity - Fundamentos de Data Science II
---
### Módulo Data Wrangle

#### Suzana Barboza

# Projeto We Rate Dogs
---


Projeto com ênfase na prática do aprendizado de Data Wrangle. Feito na linguagem Python.
É feita a coleta de 3 base de dados distintas e a limpeza das mesmas. Após isso, é feita a junção dos dados de acordo com a lei dos dados arrumados.
Para a conclusão é feita visualizações da base de dados final


## 1. Gather 

In [423]:
#importação de bibliotecas necessárias
import pandas as pd
import numpy as np
import requests
import tweepy
import json
from datetime import datetime as dt

### 1.1 CSV

Busca do dataframe na planilha fornecida pela Udacity

In [424]:
df_wrd = pd.read_csv('twitter-archive-enhanced.csv') #dataframe we rate dogs

### 1.2 TSV

Busca o arquivo na url fornecida pela Udacity e armazena em um dataframe


In [425]:
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
response = requests.get(url)
rows  = response.text.split('\n')
#deleta última linha em branco
del rows[len(rows)-1]
#separa as colunas de acordo com a tabulação
for i in range(len(rows)):
    rows[i] = rows[i].split('\t')
header = rows[0]
#deleta o header
del rows[0]
df_prediction = pd.DataFrame(np.array(rows), columns=header)

### 1.3 Twitter

Buscar informações de tweets do dataframe 'we rate dogs' através da API e armazena em um arquivo de texto nomeado 'tweet_json.txt'.

Após a coleta o armazenamento desse arquivo, ele é lido e armazenado em um dataframe

In [426]:
# Pretty Print JSON method
def pp_json(json_thing, sort=True, indents=4):
    if type(json_thing) is str:
        print(json.dumps(json.loads(json_thing), sort_keys=sort, indent=indents))
    else:
        print(json.dumps(json_thing, sort_keys=sort, indent=indents))
    return None

def default_json(o):
    if isinstance(o, np.int64): return int(o)  
    raise TypeError
    
#Função que faz a busca no twitter  das informações do dataframe provido pelo parâmetro df_wrd e cria o arquivo txt em formato json   
def getInfoFromTwitter(df_wrd, consumerkey, consumer_secret, access_token, access_token_secret):
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
    ids = df_wrd.tweet_id
    tweet_info = {}
    erros_count = 0
    print("Início: ", dt.now().time())
    for i in ids:    
        try:
            status = api.get_status(i) 
            tweet_info[i] = {'favorite_count': status.favorite_count, 'retweet_count':status.retweet_count }           
        except tweepy.TweepError as error:
            print("Erro.. ", error, " ", dt.now().time())        
            erros_count +=1        
    print("Total: ", len(ids))
    print("Erros: ", erros_count)
    print("Fim: ", dt.now().time())
    #write json
    with open('tweet_json.txt', 'w') as outfile:  
        json.dump(tweet_info, outfile, default=default_json)

In [427]:
#chama o método para criação do arquivo tweet_json.txt
consumer_key = ''
consumer_secret = ''
access_token = ''
access_token_secret = ''
#getInfoFromTwitter(df_wrd, consumer_key, consumer_secret, access_token, access_token_secret)

In [428]:
#lê o arquivo tweet_json.txt e armazena em um dataframe
with open('tweet_json.txt') as file:
    twitter_data = json.load(file)
data = []
for key in twitter_data.keys():
    data.append([key, twitter_data[key]['favorite_count'], twitter_data[key]['retweet_count']])
header = ['tweet_id', 'favorite_count', 'retweet_count']
df_twitter = pd.DataFrame(np.array(data), columns=header)

## 2. Access

Item destinado ao entendimento dos dataframes para identificação de problemas

In [429]:
df_wrd.sample(5)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
1483,693262851218264065,NaN,NaN,2016-01-30 02:41:58 +0000,"<a href=""http://twitter.com/download/iphone"" r...",I hope you guys enjoy this beautiful snowy pup...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/693262851...,11,10,None,None,None,pupper,None
1663,682808988178739200,6.827884e+17,4.196984e+09,2016-01-01 06:22:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...","I'm aware that I could've said 20/16, but here...",NaN,NaN,NaN,NaN,20,16,None,None,None,None,None
366,828801551087042563,NaN,NaN,2017-02-07 03:04:22 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Gus. He likes to be clo...,8.102541e+17,4.196984e+09,2016-12-17 22:43:27 +0000,https://twitter.com/dog_rates/status/810254108...,12,10,Gus,None,None,None,None
2145,669926384437997569,NaN,NaN,2015-11-26 17:11:11 +0000,"<a href=""http://twitter.com/download/iphone"" r...",I've never seen a dog so genuinely happy about...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/669926384...,12,10,None,None,None,None,None
1922,674255168825880576,NaN,NaN,2015-12-08 15:52:13 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Say hello to Bobbay. He's a marshmallow wizard...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/674255168...,10,10,Bobbay,None,None,None,None


In [430]:
df_wrd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [431]:
df_wrd.describe()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,retweeted_status_id,retweeted_status_user_id,rating_numerator,rating_denominator
count,2.356000e+03,7.800000e+01,7.800000e+01,1.810000e+02,1.810000e+02,2356.000000,2356.000000
mean,7.427716e+17,7.455079e+17,2.014171e+16,7.720400e+17,1.241698e+16,13.126486,10.455433
std,6.856705e+16,7.582492e+16,1.252797e+17,6.236928e+16,9.599254e+16,45.876648,6.745237
min,6.660209e+17,6.658147e+17,1.185634e+07,6.661041e+17,7.832140e+05,0.000000,0.000000
25%,6.783989e+17,6.757419e+17,3.086374e+08,7.186315e+17,4.196984e+09,10.000000,10.000000
50%,7.196279e+17,7.038708e+17,4.196984e+09,7.804657e+17,4.196984e+09,11.000000,10.000000
75%,7.993373e+17,8.257804e+17,4.196984e+09,8.203146e+17,4.196984e+09,12.000000,10.000000
max,8.924206e+17,8.862664e+17,8.405479e+17,8.874740e+17,7.874618e+17,1776.000000,170.000000


In [432]:
df_wrd[df_wrd.in_reply_to_status_id.notna()]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
30,886267009285017600,8.862664e+17,2.281182e+09,2017-07-15 16:51:35 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@NonWhiteHat @MayhewMayhem omg hello tanner yo...,NaN,NaN,NaN,NaN,12,10,None,None,None,None,None
55,881633300179243008,8.816070e+17,4.738443e+07,2017-07-02 21:58:53 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@roushfenway These are good dogs but 17/10 is ...,NaN,NaN,NaN,NaN,17,10,None,None,None,None,None
64,879674319642796034,8.795538e+17,3.105441e+09,2017-06-27 12:14:36 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@RealKentMurphy 14/10 confirmed,NaN,NaN,NaN,NaN,14,10,None,None,None,None,None
113,870726314365509632,8.707262e+17,1.648776e+07,2017-06-02 19:38:25 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@ComplicitOwl @ShopWeRateDogs &gt;10/10 is res...,NaN,NaN,NaN,NaN,10,10,None,None,None,None,None
148,863427515083354112,8.634256e+17,7.759620e+07,2017-05-13 16:15:35 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@Jack_Septic_Eye I'd need a few more pics to p...,NaN,NaN,NaN,NaN,12,10,None,None,None,None,None
149,863079547188785154,6.671522e+17,4.196984e+09,2017-05-12 17:12:53 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Ladies and gentlemen... I found Pipsy. He may ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/863079547...,14,10,None,None,None,None,None
179,857214891891077121,8.571567e+17,1.806710e+08,2017-04-26 12:48:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@Marc_IRL pixelated af 12/10,NaN,NaN,NaN,NaN,12,10,None,None,None,None,None
184,856526610513747968,8.558181e+17,4.196984e+09,2017-04-24 15:13:52 +0000,"<a href=""http://twitter.com/download/iphone"" r...","THIS IS CHARLIE, MARK. HE DID JUST WANT TO SAY...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/856526610...,14,10,None,None,None,None,None
186,856288084350160898,8.562860e+17,2.792810e+08,2017-04-23 23:26:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@xianmcguire @Jenna_Marbles Kardashians wouldn...,NaN,NaN,NaN,NaN,14,10,None,None,None,None,None
188,855862651834028034,8.558616e+17,1.943518e+08,2017-04-22 19:15:32 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@dhmontgomery We also gave snoop dogg a 420/10...,NaN,NaN,NaN,NaN,420,10,None,None,None,None,None


In [433]:
df_prediction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null object
jpg_url     2075 non-null object
img_num     2075 non-null object
p1          2075 non-null object
p1_conf     2075 non-null object
p1_dog      2075 non-null object
p2          2075 non-null object
p2_conf     2075 non-null object
p2_dog      2075 non-null object
p3          2075 non-null object
p3_conf     2075 non-null object
p3_dog      2075 non-null object
dtypes: object(12)
memory usage: 194.6+ KB


In [434]:
df_prediction.describe()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
count,2075,2075,2075,2075,2075,2075,2075,2075,2075,2075,2075,2075
unique,2075,2009,4,378,2006,2,405,2004,2,408,2006,2
top,882762694511734784,https://pbs.twimg.com/media/CdHwZd0VIAA4792.jpg,1,golden_retriever,0.796313,True,Labrador_retriever,0.0693617,True,Labrador_retriever,0.000917414,True
freq,1,2,1780,150,2,1532,104,3,1553,79,2,1499


In [535]:
df_twitter.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2340 entries, 0 to 2339
Data columns (total 3 columns):
tweet_id          2340 non-null object
favorite_count    2340 non-null object
retweet_count     2340 non-null object
dtypes: object(3)
memory usage: 54.9+ KB


### 2.1 Problemas de Qualidade

##### Tabela We Rate Dogs

- O dataframe deve conter apenas tweets que não são retweets
- As colunas in_reply_to_status_id,in_reply_to_user_id, retweeted_status, retweeted_status_user_id  são do tipo **float** e deveriam ser do tipo **int64** 
- As colunas timestamp, retweeted_status_timestamp são do tipo **object** e deveriam ser do tipo **datetime**
- As colunas rating_numerator e rating_denomitator possuem outliers significativos levando em conta o máximo, a média e o percentis.
- As colunas doggo, floofer, pupper, puppo deveriam ser do tipo **bool**
- A coluna rating_denominator deve ser 10, pela especificação fornecida pela Udacity
- A coluna tweet_id deve conter apenas valores únicos


##### Tabela de Predição

- A coluna tweet_id é do tipo **object** e deveria ser do tipo **int64**
- A coluna img_num é do tipo **object** e deveria ser do tipo **int**
- As colunas p1_dog, p2_dog, p3_dog são do tipo **object** e deveriam ser do tipo **bool**
- As colunas p1_conf, p2_conf, p3_conf são do tipo **object** e deveriam ser do tipo **float**
- O formato das colunas p1, p2, p3 deveriam ser padronizados

##### Tabela Twitter

- As colunas tweet_id, favorite_count, retweet_count são do tipo **object** e deveriam ser do tipo **int64


### 2.2 Problemas de Arrumação

##### Tabela We Rate Dogs

- O nome das colunas timestamp, retweeted_status_timestamp deveriam ser mais intuitivo

##### Tabela Predição
- O nome das colunas jpg_url, img_num, p1, p1_conf, p1_dog, p2, p2_conf, p2_dog, p3, p3_conf, p3_dog deveria ser mais intuitivo


## 3. Clean

Item destinado a limpeza dos dataframes em relação aos problemas identificados

In [829]:
#Copy datasets df_wrd, df_prediction, df_twitter
df_wrd_copy = df_wrd.copy()
df_prediction_copy = df_prediction.copy()
df_twitter_copy = df_twitter.copy()

df_wrd_copy.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


#### Definir

- O dataframe deve conter apenas tweets com imagens e que não são retweets

#### Programar

In [830]:
df_wrd_copy = df_wrd_copy[df_wrd_copy.in_reply_to_status_id.isna()][df_wrd_copy.in_reply_to_user_id.isna()]

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


#### Testar

In [831]:
df_wrd_copy[df_wrd_copy.in_reply_to_status_id.notna()].shape

(0, 17)

In [832]:
df_wrd_copy[df_wrd_copy.in_reply_to_user_id.notna()].shape

(0, 17)

#### Definir

- As colunas rating_numerator e rating_denomitator possuem outliers significativos levando em conta o máximo, a média e o percentis.

#### Programar

In [833]:
#coluna rating_numeretor possui 26 casos maior que 15
df_wrd_copy[df_wrd_copy.rating_numerator > 15].rating_numerator

340       75
433       84
516       24
695       75
763       27
902      165
979     1776
1120     204
1202      50
1228      99
1254      80
1274      45
1351      60
1433      44
1635     121
1712      26
1779     144
1843      88
2074     420
Name: rating_numerator, dtype: int64

In [834]:
df_wrd_copy = df_wrd_copy[df_wrd_copy.rating_numerator <= 15]

#### Testar

In [835]:
df_wrd_copy.rating_numerator.describe()

count    2259.000000
mean       10.684816
std         2.151440
min         0.000000
25%        10.000000
50%        11.000000
75%        12.000000
max        15.000000
Name: rating_numerator, dtype: float64

#### Definir

- A coluna rating_denominator deve ser 10, pela especificação fornecida pela Udacity

#### Programar

In [836]:
df_wrd_copy[df_wrd_copy.rating_denominator != 10] = 10

#### Testar

In [837]:
df_wrd_copy.rating_denominator.unique()

array([10], dtype=int64)

#### Definir

- O formato das colunas p1, p2, p3 deveriam ser padronizados

#### Programar

In [838]:
#Começar com letra maiúscula
#Separação com Espaço

df_prediction_copy.p1 = df_prediction_copy.p1.str.replace('-', ' ')
df_prediction_copy.p1 = df_prediction_copy.p1.str.replace('_', ' ')
df_prediction_copy.p1 = df_prediction_copy.p1.str.title()
df_prediction_copy.p2 = df_prediction_copy.p1.str.replace('-', ' ')
df_prediction_copy.p2 = df_prediction_copy.p1.str.replace('_', ' ')
df_prediction_copy.p2 = df_prediction_copy.p1.str.title() 
df_prediction_copy.p3 = df_prediction_copy.p1.str.replace('-', ' ')
df_prediction_copy.p3 = df_prediction_copy.p1.str.replace('_', ' ')
df_prediction_copy.p3 = df_prediction_copy.p1.str.title()

#### Testar

In [839]:
df_prediction_copy.p1.unique()

array(['Welsh Springer Spaniel', 'Redbone', 'German Shepherd',
       'Rhodesian Ridgeback', 'Miniature Pinscher',
       'Bernese Mountain Dog', 'Box Turtle', 'Chow', 'Shopping Cart',
       'Miniature Poodle', 'Golden Retriever', 'Gordon Setter',
       'Walker Hound', 'Pug', 'Bloodhound', 'Lhasa', 'English Setter',
       'Hen', 'Desktop Computer', 'Italian Greyhound', 'Maltese Dog',
       'Three Toed Sloth', 'Ox', 'Malamute', 'Guinea Pig',
       'Soft Coated Wheaten Terrier', 'Chihuahua',
       'Black And Tan Coonhound', 'Coho', 'Toy Terrier',
       'Blenheim Spaniel', 'Pembroke', 'Llama',
       'Chesapeake Bay Retriever', 'Curly Coated Retriever', 'Dalmatian',
       'Ibizan Hound', 'Border Collie', 'Labrador Retriever', 'Seat Belt',
       'Snail', 'Miniature Schnauzer', 'Airedale', 'Triceratops', 'Swab',
       'Hay', 'Hyena', 'Jigsaw Puzzle', 'West Highland White Terrier',
       'Toy Poodle', 'Giant Schnauzer', 'Vizsla', 'Vacuum', 'Rottweiler',
       'Siberian Husky', 'T

#### Definir

- O nome das colunas timestamp, retweeted_status_timestamp deveriam ser mais intuitivo

#### Programar

In [840]:
df_wrd_copy.rename(columns={'timestamp': 'created_at', 'retweeted_status_timestamp': 'retweeted_at'}, inplace=True)

#### Testar

In [841]:
df_wrd_copy.columns

Index(['tweet_id', 'in_reply_to_status_id', 'in_reply_to_user_id',
       'created_at', 'source', 'text', 'retweeted_status_id',
       'retweeted_status_user_id', 'retweeted_at', 'expanded_urls',
       'rating_numerator', 'rating_denominator', 'name', 'doggo', 'floofer',
       'pupper', 'puppo'],
      dtype='object')

#### Definir

- O nome das colunas jpg_url, img_num, p1, p1_conf, p1_dog, p2, p2_conf, p2_dog, p3, p3_conf, p3_dog deveria ser mais intuitivo

#### Programar

In [842]:
df_prediction_copy.rename(columns={'jpg_url': 'image_url', 'img_num': 'image_count', 
                                   'p1': 'first_prediction', 'p1_conf': 'first_confidence', 'p1_dog': 'first_isdog',
                                   'p2': 'second_prediction', 'p2_conf': 'second_confidence', 'p2_dog': 'second_isdog',
                                   'p3': 'third_prediction', 'p3_conf': 'third_confidence', 'p3_dog': 'third_isdog'
                                  }, inplace=True)

#### Testar

In [843]:
df_prediction_copy.columns

Index(['tweet_id', 'image_url', 'image_count', 'first_prediction',
       'first_confidence', 'first_isdog', 'second_prediction',
       'second_confidence', 'second_isdog', 'third_prediction',
       'third_confidence', 'third_isdog'],
      dtype='object')

#### Definir

- A coluna tweet_id deve conter apenas valores únicos

#### Programar

In [844]:
df_wrd_copy.drop_duplicates(inplace=True)

#### Testar

In [845]:
print("{0} | {1}".format(df_wrd_copy.shape[0], len(df_wrd_copy.tweet_id.unique())))

2255 | 2255


#### Definir

##### Tabela We Rate Dogs

- As colunas in_reply_to_status_id,in_reply_to_user_id, retweeted_status_id, retweeted_status_user_id são do tipo **float** e deveriam ser do tipo **int64**
- As colunas timestamp, retweeted_status_timestamp são do tipo **object** e deveriam ser do tipo **datetime**

##### Tabela de Predição

- A coluna tweet_id é do tipo **object** e deveria ser do tipo **int64**
- A coluna img_num é do tipo **object** e deveria ser do tipo **int**
- As colunas p1_dog, p2_dog, p3_dog são do tipo **object** e deveriam ser do tipo **bool**
- As colunas p1_conf, p2_conf, p3_conf são do tipo **object** e deveriam ser do tipo **float**


#### Programar

In [846]:
df_wrd_copy.in_reply_to_status_id = df_wrd_copy.in_reply_to_status_id.notna().astype('int64')
df_wrd_copy.in_reply_to_user_id = df_wrd_copy.in_reply_to_user_id.notna().astype('int64')
df_wrd_copy.retweeted_status_id = df_wrd_copy.retweeted_status_id.notna().astype('int64')
df_wrd_copy.retweeted_status_user_id  = df_wrd_copy.retweeted_status_user_id .notna().astype('int64')

df_wrd_copy.created_at = pd.to_datetime(df_wrd_copy.created_at, format='%Y-%m-%d %H:%M:%S')
df_wrd_copy.retweeted_at = pd.to_datetime(df_wrd_copy.retweeted_at, format='%Y-%m-%d %H:%M:%S')

df_prediction_copy.tweet_id = df_prediction_copy.tweet_id.astype('int64')
df_prediction_copy.image_count = df_prediction_copy.image_count.astype('int')

dict_bool = {'True': True, 'False': False}
df_prediction_copy.first_prediction = df_prediction_copy.first_prediction.map(dict_bool)
df_prediction_copy.second_prediction = df_prediction_copy.second_prediction.map(dict_bool)
df_prediction_copy.third_prediction = df_prediction_copy.third_prediction.map(dict_bool)

df_prediction_copy.first_confidence = df_prediction_copy.first_confidence.astype('float')
df_prediction_copy.second_confidence = df_prediction_copy.second_confidence.astype('float')
df_prediction_copy.third_confidence = df_prediction_copy.third_confidence.astype('float')

df_twitter_copy.tweet_id = df_twitter_copy.tweet_id.astype('int64')
df_twitter_copy.retweet_count = df_twitter_copy.retweet_count.astype('int64')
df_twitter_copy.favorite_count = df_twitter_copy.favorite_count.astype('int64')

#### Testar

In [847]:
df_wrd_copy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2255 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                    2255 non-null int64
in_reply_to_status_id       2255 non-null int64
in_reply_to_user_id         2255 non-null int64
created_at                  2255 non-null datetime64[ns]
source                      2255 non-null object
text                        2255 non-null object
retweeted_status_id         2255 non-null int64
retweeted_status_user_id    2255 non-null int64
retweeted_at                180 non-null datetime64[ns]
expanded_urls               2251 non-null object
rating_numerator            2255 non-null int64
rating_denominator          2255 non-null int64
name                        2255 non-null object
doggo                       2255 non-null object
floofer                     2255 non-null object
pupper                      2255 non-null object
puppo                       2255 non-null object
dtypes: datetime64[ns](2), int64(7), object(8)
memor

In [857]:
df_prediction_copy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2030 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id             2030 non-null int64
image_url            2030 non-null object
image_count          2030 non-null int32
first_prediction     0 non-null object
first_confidence     2030 non-null float64
first_isdog          2030 non-null object
second_prediction    0 non-null object
second_confidence    2030 non-null float64
second_isdog         2030 non-null object
third_prediction     0 non-null object
third_confidence     2030 non-null float64
third_isdog          2030 non-null object
dtypes: float64(3), int32(1), int64(1), object(7)
memory usage: 198.2+ KB


,tweet_id,image_url,image_count,first_prediction,first_confidence,first_isdog,second_prediction,second_confidence,second_isdog,third_prediction,third_confidence,third_isdog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,NaN,0.465074,True,NaN,0.156665,True,NaN,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,NaN,0.506826,True,NaN,0.074192,True,NaN,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,NaN,0.596461,True,NaN,0.138584,True,NaN,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,NaN,0.408143,True,NaN,0.360687,True,NaN,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,NaN,0.560311,True,NaN,0.243682,True,NaN,0.154629,True


In [849]:
df_twitter_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2340 entries, 0 to 2339
Data columns (total 3 columns):
tweet_id          2340 non-null int64
favorite_count    2340 non-null int64
retweet_count     2340 non-null int64
dtypes: int64(3)
memory usage: 54.9 KB


#### Definir

- As colunas doggo, floofer, pupper, puppo deveriam ser do tipo **bool**

#### Programar

In [850]:
dict_bool = {'True': True, 'False': False}

df_wrd_copy.doggo = df_wrd_copy.doggo.str.replace('doggo', 'True')
df_wrd_copy.doggo = df_wrd_copy.doggo.str.replace('None', 'False')
df_wrd_copy.doggo = df_wrd_copy.doggo.map(dict_bool).astype('bool')

df_wrd_copy.floofer = df_wrd_copy.floofer.str.replace('floofer', 'True')
df_wrd_copy.floofer = df_wrd_copy.floofer.str.replace('None', 'False')
df_wrd_copy.floofer = df_wrd_copy.floofer.map(dict_bool).astype('bool')


df_wrd_copy.pupper = df_wrd_copy.pupper.str.replace('pupper', 'True')
df_wrd_copy.pupper = df_wrd_copy.pupper.str.replace('None', 'False')
df_wrd_copy.pupper = df_wrd_copy.pupper.map(dict_bool).astype('bool')


df_wrd_copy.puppo = df_wrd_copy.puppo.str.replace('puppo', 'True')
df_wrd_copy.puppo = df_wrd_copy.puppo.str.replace('None', 'False')
df_wrd_copy.puppo = df_wrd_copy.puppo.map(dict_bool).astype('bool')

                                          


In [851]:
df_wrd_copy.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,created_at,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_at,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,0,0,2017-08-01 16:23:56,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,0,0,NaT,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,False,False,False,False
1,892177421306343426,0,0,2017-08-01 00:17:27,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,0,0,NaT,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,False,False,False,False
2,891815181378084864,0,0,2017-07-31 00:18:03,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,0,0,NaT,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,False,False,False,False
3,891689557279858688,0,0,2017-07-30 15:58:51,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,0,0,NaT,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,False,False,False,False
4,891327558926688256,0,0,2017-07-29 16:00:24,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,0,0,NaT,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,False,False,False,False


#### Testar

In [852]:
df_wrd_copy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2255 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                    2255 non-null int64
in_reply_to_status_id       2255 non-null int64
in_reply_to_user_id         2255 non-null int64
created_at                  2255 non-null datetime64[ns]
source                      2255 non-null object
text                        2255 non-null object
retweeted_status_id         2255 non-null int64
retweeted_status_user_id    2255 non-null int64
retweeted_at                180 non-null datetime64[ns]
expanded_urls               2251 non-null object
rating_numerator            2255 non-null int64
rating_denominator          2255 non-null int64
name                        2255 non-null object
doggo                       2255 non-null bool
floofer                     2255 non-null bool
pupper                      2255 non-null bool
puppo                       2255 non-null bool
dtypes: bool(4), datetime64[ns](2), int64(7), object(4)
memo

## 4. Merge

Como filtramos apenas tweets que não são retweets, as colunas 'retweeted_status_id', 'retweeted_status_user_id' do dataframe we dog rates não são mais necessárias

In [853]:
df_wrd_copy.drop(columns = ['retweeted_status_id','retweeted_status_user_id'], inplace=True)

In [855]:
#Criação de dataframe master
df_prediction_copy = df_prediction_copy[df_prediction_copy.tweet_id.isin(df_wrd_copy.tweet_id)]
df_twitter_copy = df_twitter_copy[df_twitter_copy.tweet_id.isin(df_wrd_copy.tweet_id)]
df = pd.merge(df_prediction_copy, df_wrd_copy, on='tweet_id')
df = pd.merge(df, df_twitter_copy, on='tweet_id')
#criação do arquivo master
df.to_csv('twitter_archive_master.csv')

## 5. Explore

In [856]:
df

,tweet_id,image_url,image_count,first_prediction,first_confidence,first_isdog,second_prediction,second_confidence,second_isdog,third_prediction,...,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo,favorite_count,retweet_count
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,NaN,0.465074,True,NaN,0.156665,True,NaN,...,https://twitter.com/dog_rates/status/666020888...,8,10,None,False,False,False,False,2537,502
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,NaN,0.506826,True,NaN,0.074192,True,NaN,...,https://twitter.com/dog_rates/status/666029285...,7,10,a,False,False,False,False,126,47
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,NaN,0.596461,True,NaN,0.138584,True,NaN,...,https://twitter.com/dog_rates/status/666033412...,9,10,a,False,False,False,False,123,43
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,NaN,0.408143,True,NaN,0.360687,True,NaN,...,https://twitter.com/dog_rates/status/666044226...,6,10,a,False,False,False,False,292,139
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,NaN,0.560311,True,NaN,0.243682,True,NaN,...,https://twitter.com/dog_rates/status/666049248...,5,10,None,False,False,False,False,107,41
5,666050758794694657,https://pbs.twimg.com/media/CT5Jof1WUAEuVxN.jpg,1,NaN,0.651137,True,NaN,0.263788,True,NaN,...,https://twitter.com/dog_rates/status/666050758...,10,10,a,False,False,False,False,130,59
6,666051853826850816,https://pbs.twimg.com/media/CT5KoJ1WoAAJash.jpg,1,NaN,0.933012,False,NaN,0.045885,False,NaN,...,https://twitter.com/dog_rates/status/666051853...,2,10,an,False,False,False,False,1203,838
7,666055525042405380,https://pbs.twimg.com/media/CT5N9tpXIAAifs1.jpg,1,NaN,0.692517,True,NaN,0.058279,True,NaN,...,https://twitter.com/dog_rates/status/666055525...,10,10,a,False,False,False,False,429,239
8,666057090499244032,https://pbs.twimg.com/media/CT5PY90WoAAQGLo.jpg,1,NaN,0.962465,False,NaN,0.014594,False,NaN,...,https://twitter.com/dog_rates/status/666057090...,9,10,a,False,False,False,False,290,139
9,666058600524156928,https://pbs.twimg.com/media/CT5Qw94XAAA_2dP.jpg,1,NaN,0.201493,True,NaN,0.192305,True,NaN,...,https://twitter.com/dog_rates/status/666058600...,8,10,the,False,False,False,False,109,57
